In [ ]:
#!pip install pymorphy2

In [ ]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import matplotlib_inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from gensim.models import Word2Vec
warnings.filterwarnings('ignore')
import gensim.downloader
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import wordcloud
from wordcloud import WordCloud
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
rus = "russian"
morph = MorphAnalyzer()
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)
vectorizer = CountVectorizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#ф-я для вывода текстовых данных в удобном формате
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)
#ф-я для удаления символов пунктуации
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in
    string.punctuation])
#ф-я для удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
#ф-я для удаления последовательностей
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text
#ф-я для удаления не буквенных символов
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text)
#ф-я токенизации с использованием Snowball

#ф-я токенизации с помощью PyMorphy2
def tokenize_morphy(text):
    sw = stopwords.words(rus)
    # добавил слово "это", "мочь", "свой", "который", "решить", "сказать"
    sw += ["это", "мочь", "свой", "который", "решить", "сказать"]
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = text.lower()
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                all_words.append("")
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        return ' '.join(all_words)
    else:
        return None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Загрузка русского датасета и объедниение
#train_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\train.jsonl'
#val_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\val.jsonl'
train_path = '/content/drive/MyDrive/course_work/data/test_ds1/train.jsonl'
val_path = '/content/drive/MyDrive/course_work/data/test_ds1/val.jsonl'
train_data = pd.read_json(train_path, lines=True)
val_data = pd.read_json(val_path, lines=True)
df = pd.concat([train_data, val_data])
df = df.drop(['label', 'hypothesis', 'verb','negation'
,'idx','no_negation'], axis=1)
df.dropna()
df.head()
df['genre'] = df['genre'].replace('kp', 'interfax')

In [ ]:
sw = stopwords.words(rus)
sw += ["это", "мочь", "свой", "который", "решить", "сказать"]

In [ ]:
Y = pd.DataFrame({'genre':[str(genre) for genre in df['genre']]})
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample)
for sample in df['premise']]})
X2 = df_morphy.dropna()
batch2 = X2.join(Y, how = 'right')
batch2 = batch2.dropna()

In [ ]:
def score_report(y_test, pred):
    acc =       accuracy_score(y_test, pred)
    recall =    recall_score(y_test, pred, average="binary", pos_label="fiction")
    precision = precision_score(y_test, pred, average="binary", pos_label="fiction")
    f1score =   f1_score(y_test, pred, average="binary", pos_label="fiction")
    scores = pd.DataFrame({"Accuracy" : [acc], "Recall" : [recall], "Precision" : [precision], "f1score" : [f1score]})
    return scores

In [ ]:
def BOW(**kwargs):
  batch = kwargs['batch']
  tst_s = kwargs['test_size']
  random_state = kwargs['random_state']
  X_train, X_test, y_train, y_test = train_test_split(batch['text'], batch['genre'], test_size = tst_s, random_state = random_state)
  vec = CountVectorizer(ngram_range=(1, 1))
  bow = vec.fit_transform(X_train.values.astype('U'))
  bow_test = vec.transform(X_test)
  scaler = MaxAbsScaler()
  bow = scaler.fit_transform(bow)
  bow_test = scaler.transform(bow_test)
  return bow, bow_test, y_train, y_test


In [ ]:
def TfIdf(**kwargs):
  batch = kwargs['batch']
  tst_s = kwargs['test_size']
  random_state = kwargs['random_state']
  X_train, X_test, y_train, y_test = train_test_split(batch['text'], batch['genre'], test_size = tst_s, random_state = random_state)
  tfidf = TfidfVectorizer(min_df = 5, max_df = 1.0, ngram_range = (1, 1))
  tfidf.fit(X_train.values.astype('U'))
  Xtrain = tfidf.transform(X_train.values.astype('U'))
  Xtest = tfidf.transform(X_test.values.astype('U'))
  return Xtrain, Xtest, y_train, y_test

In [ ]:
def W2V(**kwargs):
  batch = kwargs['batch']
  tst_s = kwargs['test_size']
  random_state = kwargs['random_state']

  sentences = batch['text'].tolist()
  sentences = [row.split() for row in sentences]

  labels = batch['genre'].tolist()

  model_w2v = gensim.models.Word2Vec(sentences, vector_size=300, window=5, min_count=1, sg=1)

  vectors = [np.mean([model_w2v.wv[word] for word in sentence if word in model_w2v.wv] or [np.zeros(100)], axis=0)
  for sentence in sentences]

  X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size = tst_s, random_state = random_state)

  return X_train, X_test, y_train, y_test


In [ ]:
def accuracy(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return accuracy_score(y_test, y_reg), accuracy_score(y_test, y_svc), accuracy_score(y_test, y_dtc), accuracy_score(y_test, y_rfc), accuracy_score(y_test, y_gbc), accuracy_score(y_test, y_abc), accuracy_score(y_test, y_knc)

In [ ]:
def precision(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return precision_score(y_test, y_reg, average="binary", pos_label="fiction"), precision_score(y_test, y_svc, average="binary", pos_label="fiction"), precision_score(y_test, y_dtc, average="binary", pos_label="fiction"), precision_score(y_test, y_rfc, average="binary", pos_label="fiction"), precision_score(y_test, y_gbc, average="binary", pos_label="fiction"), precision_score(y_test, y_abc, average="binary", pos_label="fiction"), precision_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def recall(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return recall_score(y_test, y_reg, average="binary", pos_label="fiction"), recall_score(y_test, y_svc, average="binary", pos_label="fiction"), recall_score(y_test, y_dtc, average="binary", pos_label="fiction"), recall_score(y_test, y_rfc, average="binary", pos_label="fiction"), recall_score(y_test, y_gbc, average="binary", pos_label="fiction"), recall_score(y_test, y_abc, average="binary", pos_label="fiction"), recall_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def f1(X_train, X_test, y_train, y_test):
    reg = LogisticRegression()
    reg.fit(X_train, y_train)
    y_reg = reg.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    y_svc = svc.predict(X_test)

    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    y_dtc = dtc.predict(X_test)

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_rfc = rfc.predict(X_test)

    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    y_gbc = gbc.predict(X_test)

    abc = AdaBoostClassifier()
    abc.fit(X_train, y_train)
    y_abc = abc.predict(X_test)

    knc = KNeighborsClassifier()
    knc.fit(X_train, y_train)
    y_knc = knc.predict(X_test)

    return f1_score(y_test, y_reg, average="binary", pos_label="fiction"), f1_score(y_test, y_svc, average="binary", pos_label="fiction"), f1_score(y_test, y_dtc, average="binary", pos_label="fiction"), f1_score(y_test, y_rfc, average="binary", pos_label="fiction"), f1_score(y_test, y_gbc, average="binary", pos_label="fiction"), f1_score(y_test, y_abc, average="binary", pos_label="fiction"), f1_score(y_test, y_knc, average="binary", pos_label="fiction")

In [ ]:
def statistic(X_train, X_test, y_train, y_test):
  scores = pd.DataFrame(columns = ["REG", "SVC", "DTC", "RFC", "GBC", "ABC", "KNC"],
                     index = ["ACC", "PREC", "REC", "F1"])
  acc = [reg, svc, dtc, rfc, gbc, abc, knc] = accuracy(X_train, X_test, y_train, y_test)
  scores.iloc[0, :] = acc
  prec = [reg, svc, dtc, rfc, gbc, abc, knc] = precision(X_train, X_test, y_train, y_test)
  scores.iloc[1, :] = prec
  rec = [reg, svc, dtc, rfc, gbc, abc, knc] = recall(X_train, X_test, y_train, y_test)
  scores.iloc[2, :] = rec
  f_1 = [reg, svc, dtc, rfc, gbc, abc, knc] = f1(X_train, X_test, y_train, y_test)
  scores.iloc[3, :] = f_1
  return scores


In [ ]:
from IPython.display import display

In [ ]:
random_state = 13
test_size = 0.2

Мешок слов:

In [ ]:
X_train, X_test, y_train, y_test = BOW(batch = batch2, random_state = random_state, test_size = test_size)
display(statistic(X_train, X_test, y_train, y_test))

,REG,SVC,DTC,RFC,GBC,ABC,KNC
ACC,0.854545,0.795455,0.722727,0.795455,0.790909,0.768182,0.6
PREC,0.818898,0.732877,0.725191,0.739726,0.731034,0.715278,0.563452
REC,0.920354,0.946903,0.849558,0.946903,0.938053,0.911504,0.982301
F1,0.866667,0.826255,0.811475,0.834646,0.821705,0.801556,0.716129


Tf-ifd

In [ ]:
X_train, X_test, y_train, y_test = TfIdf(batch = batch2, random_state = random_state, test_size = test_size)
display(statistic(X_train, X_test, y_train, y_test))

,REG,SVC,DTC,RFC,GBC,ABC,KNC
ACC,0.840909,0.836364,0.763636,0.822727,0.786364,0.713636,0.713636
PREC,0.795455,0.793893,0.763158,0.776119,0.736111,0.668919,0.647059
REC,0.929204,0.920354,0.778761,0.911504,0.938053,0.876106,0.973451
F1,0.857143,0.852459,0.756522,0.830645,0.824903,0.758621,0.777385


Word2Vec

In [ ]:
X_train, X_test, y_train, y_test = W2V(batch = batch2, random_state = random_state, test_size = test_size)
display(statistic(X_train, X_test, y_train, y_test))

,REG,SVC,DTC,RFC,GBC,ABC,KNC
ACC,0.518182,0.513636,0.722727,0.677273,0.677273,0.65,0.654545
PREC,0.515982,0.513636,0.727273,0.657534,0.661654,0.632353,0.617834
REC,1.0,1.0,0.743363,0.867257,0.778761,0.761062,0.858407
F1,0.680723,0.678679,0.709402,0.747082,0.710204,0.690763,0.718519


Сделать матрицу путаниц для w2v

In [ ]:
X_train, X_test, y_train, y_test = W2V(batch = batch2, random_state = random_state, test_size = test_size)
svc = SVC()
svc.fit(X_train, y_train)
y_svc = svc.predict(X_train)
y_svc

array(['fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 'fiction', 'fiction',
       'fiction', 'fiction', 'fiction', 'fiction', 

In [ ]:
y_test

['interfax',
 'fiction',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'fiction',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'interfax',
 'fiction',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'interfax',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'interfax',
 'fiction',
 'interfax',
 'interfax',
 'interfax',
 'interfax',
 'interfax',
 